In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import seaborn as sns
#% matplotlib inline
from datetime import datetime
import urllib
import json

import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.figure_factory as ff
import plotly.express as px
from plotly import tools
from plotly.subplots import make_subplots
from plotly.offline import iplot

from textblob import TextBlob, Word, Blobber
from textblob.classifiers import NaiveBayesClassifier
from textblob.taggers import NLTKTagger

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer 
from nltk.stem import WordNetLemmatizer
nltk.download('omw-1.4')
from nltk.tokenize import word_tokenize
nltk.download('maxent_ne_chunker')
nltk.download('words')

import string
import gensim
import collections
from gensim.corpora import Dictionary
import difflib 

from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel
from gensim.models import Word2Vec
from sklearn.decomposition import PCA

from autocorrect import Speller

from bs4 import BeautifulSoup
import requests
import csv
import time

import requests 
import datetime as dt
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt

import re
import string
import gc
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/aditidutta/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/aditidutta/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/aditidutta/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
# Parameters
query = '(misogyny OR misogynist OR sexism OR sexist OR gender OR "online hate" OR "online abuse" OR "online harassment") AND ("social media" OR "social platform" OR twitter OR Twitter OR Facebook OR facebook OR reddit OR Reddit OR instagram OR Instagram OR meta OR linkedin OR LinkedIn OR youtube OR YouTube)' 

# original query only gave 22 results. This gives 406 results
max_results = 1000

#Call arXiv API with the query parameters
url = f'http://export.arxiv.org/api/query?search_query={query}&max_results={max_results}'
resp = requests.get(url)

ns = { 'r':'http://www.w3.org/2005/Atom'}
root = ET.fromstring(resp.text)

# Construct the dataframe from the response
all_papers = list()
entries = root.findall('r:entry',namespaces=ns)
for entry in entries :
    all_papers.append({l.tag[l.tag.index('}')+1:] :l.text for l in entry})

In [3]:
len(all_papers)

435

In [4]:
df = pd.DataFrame(all_papers)
df

,id,updated,published,title,summary,author,doi,link,journal_ref,primary_category,category,comment
0,http://arxiv.org/abs/2106.08409v1,2021-06-15T20:01:28Z,2021-06-15T20:01:28Z,Benchmark dataset of memes with text transcrip...,In this paper we present a benchmark dataset...,\n,10.1016/j.dib.2022.108526,None,Data in brief 44 (2022): 108526,None,None,NaN
1,http://arxiv.org/abs/1703.04791v2,2017-03-21T21:31:04Z,2017-03-14T22:37:01Z,Wearing Many (Social) Hats: How Different are ...,This paper investigates when users create pr...,\n,NaN,None,NaN,None,None,Accepted at the 11th International AAAI Confer...
2,http://arxiv.org/abs/2103.10195v1,2021-03-18T12:01:13Z,2021-03-18T12:01:13Z,Let-Mi: An Arabic Levantine Twitter Dataset fo...,Online misogyny has become an increasing wor...,\n,NaN,None,NaN,None,None,"10 pages, 2 figures, WANLP 2021 co-located wit..."
3,http://arxiv.org/abs/1902.10584v1,2019-02-27T15:26:45Z,2019-02-27T15:26:45Z,When a Tweet is Actually Sexist. A more Compre...,Sexism is very common in social media and ma...,\n,NaN,None,NaN,None,None,NaN
4,http://arxiv.org/abs/1507.02770v1,2015-07-10T02:58:04Z,2015-07-10T02:58:04Z,Ang Social Network sa Facebook ng mga Taga-Bat...,Online social networking (OSN) has become of...,\n,NaN,None,Asia Pacific Journal of Multidisciplinary Rese...,None,None,"16 pages, 8 figures; The text is written in Fi..."
...,...,...,...,...,...,...,...,...,...,...,...,...
430,http://arxiv.org/abs/2208.01182v1,2022-08-02T00:22:20Z,2022-08-02T00:22:20Z,Mitigating Biases in Student Performance Predi...,Traditional learning-based approaches to stu...,\n,NaN,None,NaN,None,None,"10 pages, CIKM 2022"
431,http://arxiv.org/abs/2201.10182v1,2022-01-25T08:56:14Z,2022-01-25T08:56:14Z,Pre-Trained Language Transformers are Universa...,Facial images disclose many hidden personal ...,\n,NaN,None,NaN,None,None,NaN
432,http://arxiv.org/abs/2205.11264v2,2022-05-26T15:45:50Z,2022-05-20T15:29:38Z,Adaptive Fairness-Aware Online Meta-Learning f...,The fairness-aware online learning framework...,\n,NaN,None,NaN,None,None,Proceedings of the 28th ACM SIGKDD Conference ...
433,http://arxiv.org/abs/1604.02620v1,2016-04-10T00:14:43Z,2016-04-10T00:14:43Z,Surveying Turkish high school and university s...,Student attitudes and approaches to problem ...,\n,10.1103/PhysRevPhysEducRes.12.010129,None,"Physical Review Physics Education Research, 12...",None,None,"16 pages, Keywords: Physics Education Research..."


In [5]:
def url_encode(text):
    text = urllib.parse.quote(text)
    return text

In [ ]:
list_titles = df['title'].apply(url_encode)
# Store it as a list
list_titles = list_titles.to_list()

In [9]:
# Automated, but without the API credentials
limit= 1 # Because we only want exact match
json_data = []
print('Starting to collect results')

for title in list_titles:
    # Make the request
    print(f'{title}')
    responses = requests.get(f'https://api.semanticscholar.org/graph/v1/paper/search?query=title:({title})&limit={limit}', stream = True)
    json = responses.json()
    # Check if the sum of offset and limit exceeds 10000 or if any values are left
    # To avoid {'error': 'offset + limit must be < 10000'} Why? Check the About.md for answers
    #if 'data' in json: 
    if ('data' not in json) or (not json['data']):
        print('Going to the next title')
        continue
    else:
        paper_id = json['data'][0]['paperId']
        responses = requests.get(f'https://api.semanticscholar.org/graph/v1/paper/{paper_id}?fields=url,abstract,authors,title,venue,year,publicationTypes,referenceCount,citationCount,influentialCitationCount,isOpenAccess,fieldsOfStudy,tldr', stream = True)
        json = responses.json()
        if 'title' in json: 
            # The next step is needed. Otherwise will give unhashable dict error
            json_extension = {}
            # Add all the different kinds of objects which are of interest
            json_extension['Title'] = json['title']
            json_extension['Authors'] = json['authors']
            json_extension['Abstract'] = json['abstract']
            json_extension['Venue'] = json['venue']
            json_extension['Year'] = json['year']
            json_extension['Publication_Type'] = json['publicationTypes']
            json_extension['Reference_Count'] = json['referenceCount']
            json_extension['Citation_Count'] = json['citationCount']
            json_extension['Influential_Citation_Count'] = json['influentialCitationCount']
            json_extension['Open_Access'] = json['isOpenAccess']
            json_extension['Discipline'] = json['fieldsOfStudy']
            json_extension['TLDR'] = json['tldr']
            json_data.append(json_extension)
            time.sleep(1)
            print(f'Results of {paper_id} started')
        else:
            print(f'Endpoint request timed out or no data could be retrieved')
            continue


Starting to collect results
Benchmark%20dataset%20of%20memes%20with%20text%20transcriptions%20for%20automatic%0A%20%20detection%20of%20multi-modal%20misogynistic%20content
Results of 231a8c3a8851e4a04cecd6b0e2c257620aabd34c started
Wearing%20Many%20%28Social%29%20Hats%3A%20How%20Different%20are%20Your%20Different%20Social%0A%20%20Network%20Personae%3F
Results of 9873220aa92df2908830bf18681ebf98b410b560 started
Let-Mi%3A%20An%20Arabic%20Levantine%20Twitter%20Dataset%20for%20Misogynistic%20Language
Going to the next title
When%20a%20Tweet%20is%20Actually%20Sexist.%20A%20more%20Comprehensive%20Classification%20of%0A%20%20Different%20Online%20Harassment%20Categories%20and%20The%20Challenges%20in%20NLP
Results of 295505a820d8f5569c7228e1dfea3778666adcb1 started
Ang%20Social%20Network%20sa%20Facebook%20ng%20mga%20Taga-Batangas%20at%20ng%20mga%0A%20%20Taga-Laguna%3A%20Isang%20Paghahambing
Going to the next title
%23ILookLikeAnEngineer%3A%20Using%20Social%20Media%20Based%20Hashtag%20Activism%0

In [10]:
df = pd.DataFrame(json_data)
df

,Title,Authors,Abstract,Venue,Year,Publication_Type,Reference_Count,Citation_Count,Influential_Citation_Count,Open_Access,Discipline,TLDR
0,Benchmark dataset of memes with text transcrip...,"[{'authorId': '2032578', 'name': 'F. Gasparini...",None,Data in Brief,2021,[JournalArticle],12,8,0,False,"[Medicine, Computer Science]","{'model': 'tldr@v2.0.0', 'text': 'A benchmark ..."
1,Wearing Many (Social) Hats: How Different Are ...,"[{'authorId': '36957299', 'name': 'Changtao Zh...",\n \n This paper investigates when users creat...,International Conference on Web and Social Media,2017,[JournalArticle],50,26,3,True,"[Computer Science, Psychology]","{'model': 'tldr@v2.0.0', 'text': 'It is shown ..."
2,When a Tweet is Actually Sexist. A more Compre...,"[{'authorId': '10745415', 'name': 'Sima Sharif...",Sexism is very common in social media and make...,ArXiv,2019,[JournalArticle],17,18,0,False,[Computer Science],"{'model': 'tldr@v2.0.0', 'text': 'This paper p..."
3,#ILookLikeAnEngineer: Using Social Media Based...,"[{'authorId': '145539521', 'name': 'Aqdas Mali...","Each year, significant investment of time and ...",ArXiv,2018,[JournalArticle],24,5,0,False,"[Computer Science, Sociology]","{'model': 'tldr@v2.0.0', 'text': 'It is argued..."
4,The ComMA Dataset V0.2: Annotating Aggression ...,"[{'authorId': '2142831715', 'name': 'Ritesh Ku...","In this paper, we discuss the development of a...",International Conference on Language Resources...,2021,[JournalArticle],45,12,1,False,[Computer Science],"{'model': 'tldr@v2.0.0', 'text': 'The paper gi..."
...,...,...,...,...,...,...,...,...,...,...,...,...
278,Adaptive image-feature learning for disease cl...,"[{'authorId': '80843089', 'name': 'Hendrik Bur...",None,International Conference on Medical Image Comp...,2019,[JournalArticle],13,11,0,True,"[Computer Science, Engineering, Mathematics]","{'model': 'tldr@v2.0.0', 'text': 'This work pr..."
279,Decision Support for Intoxication Prediction U...,"[{'authorId': '80843089', 'name': 'Hendrik Bur...",None,International Conference on Medical Image Comp...,2020,[JournalArticle],17,1,0,True,[Computer Science],"{'model': 'tldr@v2.0.0', 'text': 'A new machin..."
280,Mitigating Biases in Student Performance Predi...,"[{'authorId': '9628638', 'name': 'Yun-Wei Chu'...",Traditional learning-based approaches to stude...,International Conference on Information and Kn...,2022,"[JournalArticle, Book, Conference]",50,2,1,True,[Computer Science],"{'model': 'tldr@v2.0.0', 'text': 'This paper p..."
281,Pre-Trained Language Transformers are Universa...,"[{'authorId': '2061600973', 'name': 'Rahul Goe...",Facial images disclose many hidden personal tr...,ArXiv,2022,[JournalArticle],46,1,0,False,[Computer Science],"{'model': 'tldr@v2.0.0', 'text': 'A novel meth..."


In [12]:
# remove the ones with no disciplines
df= df[~df.Discipline.isna()]
df = df.reset_index(drop=True)
df['index'] = df.index
df

,Title,Authors,Abstract,Venue,Year,Publication_Type,Reference_Count,Citation_Count,Influential_Citation_Count,Open_Access,Discipline,TLDR,index
0,Benchmark dataset of memes with text transcrip...,"[{'authorId': '2032578', 'name': 'F. Gasparini...",None,Data in Brief,2021,[JournalArticle],12,8,0,False,"[Medicine, Computer Science]","{'model': 'tldr@v2.0.0', 'text': 'A benchmark ...",0
1,Wearing Many (Social) Hats: How Different Are ...,"[{'authorId': '36957299', 'name': 'Changtao Zh...",\n \n This paper investigates when users creat...,International Conference on Web and Social Media,2017,[JournalArticle],50,26,3,True,"[Computer Science, Psychology]","{'model': 'tldr@v2.0.0', 'text': 'It is shown ...",1
2,When a Tweet is Actually Sexist. A more Compre...,"[{'authorId': '10745415', 'name': 'Sima Sharif...",Sexism is very common in social media and make...,ArXiv,2019,[JournalArticle],17,18,0,False,[Computer Science],"{'model': 'tldr@v2.0.0', 'text': 'This paper p...",2
3,#ILookLikeAnEngineer: Using Social Media Based...,"[{'authorId': '145539521', 'name': 'Aqdas Mali...","Each year, significant investment of time and ...",ArXiv,2018,[JournalArticle],24,5,0,False,"[Computer Science, Sociology]","{'model': 'tldr@v2.0.0', 'text': 'It is argued...",3
4,The ComMA Dataset V0.2: Annotating Aggression ...,"[{'authorId': '2142831715', 'name': 'Ritesh Ku...","In this paper, we discuss the development of a...",International Conference on Language Resources...,2021,[JournalArticle],45,12,1,False,[Computer Science],"{'model': 'tldr@v2.0.0', 'text': 'The paper gi...",4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,Adaptive image-feature learning for disease cl...,"[{'authorId': '80843089', 'name': 'Hendrik Bur...",None,International Conference on Medical Image Comp...,2019,[JournalArticle],13,11,0,True,"[Computer Science, Engineering, Mathematics]","{'model': 'tldr@v2.0.0', 'text': 'This work pr...",275
276,Decision Support for Intoxication Prediction U...,"[{'authorId': '80843089', 'name': 'Hendrik Bur...",None,International Conference on Medical Image Comp...,2020,[JournalArticle],17,1,0,True,[Computer Science],"{'model': 'tldr@v2.0.0', 'text': 'A new machin...",276
277,Mitigating Biases in Student Performance Predi...,"[{'authorId': '9628638', 'name': 'Yun-Wei Chu'...",Traditional learning-based approaches to stude...,International Conference on Information and Kn...,2022,"[JournalArticle, Book, Conference]",50,2,1,True,[Computer Science],"{'model': 'tldr@v2.0.0', 'text': 'This paper p...",277
278,Pre-Trained Language Transformers are Universa...,"[{'authorId': '2061600973', 'name': 'Rahul Goe...",Facial images disclose many hidden personal tr...,ArXiv,2022,[JournalArticle],46,1,0,False,[Computer Science],"{'model': 'tldr@v2.0.0', 'text': 'A novel meth...",278


## Authors

In [13]:
authors = pd.json_normalize(df['Authors'])
print("Number of null values in authors: \n First author: {first}\n Second author: {second} \n Third author: {third} \n Fourth author: {fourth}\n Fifth author: {fifth}\n Sixth author: {sixth}".format(first=authors[0].isna().sum(), second= authors[1].isna().sum(), third= authors[2].isna().sum(), fourth= authors[3].isna().sum(), fifth= authors[4].isna().sum(), sixth= authors[5].isna().sum()))

Number of null values in authors: 
 First author: 0
 Second author: 19 
 Third author: 64 
 Fourth author: 132
 Fifth author: 186
 Sixth author: 226


In [14]:
nested_authors1 = pd.json_normalize(authors[0]).rename(columns={'authorId':'author_1_id', 'name':'author_1_name'})
nested_authors1['index'] = nested_authors1.index
nested_authors1_comb = nested_authors1.groupby(['author_1_id','author_1_name']).size().reset_index().rename(columns={0:'count'})
nested_authors1_comb = nested_authors1_comb.sort_values(by='count', ascending=False)

# Following the same technique from the previous author 1 set
# Author 2
nested_authors2 = pd.json_normalize(authors[1]).rename(columns={'authorId':'author_2_id', 'name':'author_2_name'})
nested_authors2['index'] = nested_authors2.index
nested_authors2_comb = nested_authors2.groupby(['author_2_id','author_2_name']).size().reset_index().rename(columns={0:'count'})
nested_authors2_comb = nested_authors2_comb.sort_values(by='count', ascending=False)

# Author 3
nested_authors3 = pd.json_normalize(authors[2]).rename(columns={'authorId':'author_3_id', 'name':'author_3_name'})
nested_authors3['index'] = nested_authors3.index
nested_authors3_comb = nested_authors3.groupby(['author_3_id','author_3_name']).size().reset_index().rename(columns={0:'count'})
nested_authors3_comb = nested_authors3_comb.sort_values(by='count', ascending=False)

# Author 4
nested_authors4 = pd.json_normalize(authors[3]).rename(columns={'authorId':'author_4_id', 'name':'author_4_name'})
nested_authors4['index'] = nested_authors4.index
nested_authors4_comb = nested_authors4.groupby(['author_4_id','author_4_name']).size().reset_index().rename(columns={0:'count'})
nested_authors4_comb = nested_authors4_comb.sort_values(by='count', ascending=False)

# Author 5
nested_authors5 = pd.json_normalize(authors[4]).rename(columns={'authorId':'author_5_id', 'name':'author_5_name'})
nested_authors5['index'] = nested_authors5.index
nested_authors5_comb = nested_authors5.groupby(['author_5_id','author_5_name']).size().reset_index().rename(columns={0:'count'})
nested_authors5_comb = nested_authors5_comb.sort_values(by='count', ascending=False)

# Author 6
nested_authors6 = pd.json_normalize(authors[5]).rename(columns={'authorId':'author_6_id', 'name':'author_6_name'})
nested_authors6['index'] = nested_authors6.index
nested_authors6_comb = nested_authors6.groupby(['author_6_id','author_6_name']).size().reset_index().rename(columns={0:'count'})
nested_authors6_comb = nested_authors6_comb.sort_values(by='count', ascending=False)

In [15]:
#only run once
df = pd.merge(df, nested_authors1, on="index")
df = pd.merge(df, nested_authors2, on="index")
df = pd.merge(df, nested_authors3, on="index")
df = pd.merge(df, nested_authors4, on="index")
df = pd.merge(df, nested_authors5, on="index")
df = pd.merge(df, nested_authors6, on="index")

In [23]:
df = df.drop(columns=['Authors'])

## Fields 

In [16]:
fields = df.Discipline.copy()

fields = fields.explode()
fields_index = fields.index.to_list()
fields_name = fields.to_list()
fields = [fields_index, fields_name]
fields = pd.DataFrame(fields).transpose()
fields.columns = ['index', 'name']

# unique_fields = list(fields.name.unique()) # Will be useful later

fields = fields.groupby(by='index')['name'].apply(lambda x:x.str.cat(sep=", ")).reset_index(drop=True)
fields = pd.DataFrame(fields)
fields['index'] = fields.index

fields

,name,index
0,"Medicine, Computer Science",0
1,"Computer Science, Psychology",1
2,Computer Science,2
3,"Computer Science, Sociology",3
4,Computer Science,4
...,...,...
275,"Computer Science, Engineering, Mathematics",275
276,Computer Science,276
277,Computer Science,277
278,Computer Science,278


In [17]:
fields_type = fields['name'].to_list()
unique_fields = []
for i in fields_type:
    i = i.split(", ")
    for j in i:
        if j not in unique_fields:
            if j != '':
                unique_fields.append(j)
        else:
            continue

unique_fields

['Medicine',
 'Computer Science',
 'Psychology',
 'Sociology',
 'Mathematics',
 'Political Science',
 'Geography',
 'Physics',
 'Business',
 'Economics',
 'History',
 'Engineering',
 'Biology']

In [18]:
# drop the column 'fieldsOfStudy'
df = df.drop(columns=['Discipline'])

# merge the above dataframe to this one
df = pd.merge(df, fields, on="index").rename(columns={'name': 'fieldsOfStudy'}) # drops the rows with null values

## Publication Type

In [19]:
pub_type = df['Publication_Type'].copy()

pub_type = pub_type.explode()
pub_type_index = pub_type.index.to_list()
pub_type_name = pub_type.to_list()
pub_type = [pub_type_index, pub_type_name]
pub_type = pd.DataFrame(pub_type).transpose()
pub_type.columns = ['index', 'name']

# unique_fields = list(fields.name.unique()) # Will be useful later

pub_type = pub_type.groupby(by='index')['name'].apply(lambda x:x.str.cat(sep=", ")).reset_index(drop=True)
pub_type = pd.DataFrame(pub_type)
pub_type['index'] = pub_type.index

pub_type

,name,index
0,JournalArticle,0
1,JournalArticle,1
2,JournalArticle,2
3,JournalArticle,3
4,JournalArticle,4
...,...,...
275,JournalArticle,275
276,JournalArticle,276
277,"JournalArticle, Book, Conference",277
278,JournalArticle,278


In [20]:
pub_type_cat = pub_type['name'].to_list()
unique_fields = []
for i in pub_type_cat:
    i = i.split(", ")
    for j in i:
        if j not in unique_fields:
            if j != '':
                unique_fields.append(j)
        else:
            continue

unique_fields

['JournalArticle', 'Book', 'Review', 'Conference', 'MetaAnalysis']

In [21]:
# drop the column 'fieldsOfStudy'
df = df.drop(columns=['Publication_Type'])

# merge the above dataframe to this one
df = pd.merge(df, pub_type, on="index").rename(columns={'name': 'Publication_Type'}) # drops the rows with null values

In [24]:
# df.to_csv('collected_data.csv')

In [25]:
# Create two dataframes based on Computer Science and Social Science

# All computer science and related fields
df_comp = df[df.fieldsOfStudy.str.contains('Computer Science') == True]
df_engg = df[df.fieldsOfStudy.str.contains('Engineering') == True]
df_maths = df[df.fieldsOfStudy.str.contains('Mathematics') == True]
df_comp = pd.concat([df_comp, df_engg, df_maths])
df_comp = df_comp.drop_duplicates(subset=['index']).reset_index(drop=True) #dropping rows with same index values

# All social science and related fields
df_soc = df[df.fieldsOfStudy.str.contains('Sociology') == True]
df_pol = df[df.fieldsOfStudy.str.contains('Political Science') == True]
df_phy = df[df.fieldsOfStudy.str.contains('Philosophy') == True]
df_psc = df[df.fieldsOfStudy.str.contains('Psychology') == True]
df_soc = pd.concat([df_soc, df_pol, df_phy, df_psc])
df_soc = df_soc.drop_duplicates(subset=['index']).reset_index(drop=True) #dropping rows with same index values

In [26]:
len(df_comp), len(df_soc)

(274, 63)